In [1]:
from datetime import datetime

import numpy as np

import os

import pandas as pd

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

from time import time, gmtime, strftime

from bert.tokenization.bert_tokenization import FullTokenizer

from utils.models.modelUtils import loadMeanFromFile, calcAccConcatModel

from final_models import create_text_model, create_model_meta, buildConcatModelTitleMeta

from utils.callbacks.MyCallbacks import MyCallbacks
from utils.datagenUtils.dual_modal.DataSeqTitleMeta import DataSequenceTitleMeta
from utils.datagenUtils.datagenUtils import checkDataframe

from utils.telegramUtils.telegram_bot import telegram_send_message
from utils.textUtils.textpreprocessing import FakeDetectionDataTrainVal, FakeDetectionDataTest
from utils.textUtils.commentsProcessing import FakeDetectionDataCommentsTest, FakeDetectionDataCommentsTrainVal

from utils.callbacks.callbackUtils import plotTimesPerEpoch

from utils.fileDirUtils.fileDirUtils import createDirIfNotExists

from sklearn.metrics import accuracy_score


In [2]:
#Verbose settings:
verbose = False
TF_VERBOSE = 1 # 1 = Progress bar 2 = one line per epoch only!
TF_DETERMINISTIC_OPS = 1 # Makes everything also on GPU deterministic

# Classes:
NUM_CLASS = 2  # FAKE | NO FAKE

# Hyperparameters
GLOBAL_BATCH_SIZE = 4096
EPOCHS = 15

# Optimizer parameters:
# Adam
LEARNING_RATE = 1e-5
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-8

#optimizers:

optimizer = Adam(LEARNING_RATE, BETA_1, BETA_2, EPSILON)

# Bert Parameters
MAX_SEQUENCE_LENGTH = 128 # from model definition!

# Training switches
RETRAIN_WHOLE_MODEL = False
loadPretrained = False # Attention that model weights path is set !

# Custom telegram send text 
CUSTOM_TEXT = f'Batch Size: {GLOBAL_BATCH_SIZE}, Epochs: {EPOCHS}, Optimizer: Adam, Learning Rate; {LEARNING_RATE}, Beta_1: {BETA_1}, Beta_2: {BETA_2}, Epsilon: {EPSILON}, BERT Max sequence length: {MAX_SEQUENCE_LENGTH}'


telegram_send_message(f'-----------------START-----------------')
print('START')
print(CUSTOM_TEXT)
telegram_send_message(CUSTOM_TEXT)

START
Batch Size: 4096, Epochs: 10, Optimizer: Adam, Learning Rate; 1e-05, Beta_1: 0.9, Beta_2: 0.999, Epsilon: 1e-08, BERT Max sequence length: 128


In [3]:
pathToRootModelDir = '/home/armin/repos/fkd-model-handling/models/best_models'

pathToBertModelTitle = os.path.join(pathToRootModelDir, 'single_text_title', 'weights-improvement-02-0.88.hdf5')
pathToMetaModel = os.path.join(pathToRootModelDir, 'single_meta', 'weights-improvement-100-0.62.hdf5')

pathToCSVWithFileNamesAndLabelsTrain = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/train_text_image_meta_label.csv'

pathToCSVWithFileNamesAndLabelsVal = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/val_text_image_meta_label.csv'

pathToCSVWithFileNamesAndLabelsTest = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/test_text_image_meta_label.csv'

checkpointDir = '/home/armin/repos/FKD-Dataset/011_checkpoints'

titleMetaModelWeightsPath = ""

root = '/home/armin/repos/fkd-model-handling/'
bert_model_dir = os.path.join(root, 'multi_cased_L-12_H-768_A-12')
bert_config_file = os.path.join(bert_model_dir, "bert_config.json")
bert_ckpt_file = os.path.join(bert_model_dir, "bert_model.ckpt") 

In [4]:
# Other ettings

# Time settings:
current_time = datetime.now().strftime("%Y-%m-%d_%H:%M")

#Checkpoint settings:
checkpoint_name = "model_concat_title_meta"

checkpointDir = os.path.join(checkpointDir, (checkpoint_name + '_' + current_time))

fileName="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filePath = os.path.join(checkpointDir, fileName)

In [5]:
# Callback Handling:
tensorboardDir = os.path.join(checkpointDir, 'tensorboard')

createDirIfNotExists(tensorboardDir)
createDirIfNotExists(checkpointDir)


callbacks_list = MyCallbacks(tensorboardDir, filePath, earlyStopping=True).createCheckpoints()

Directory /home/armin/repos/FKD-Dataset/011_checkpoints/model_concat_title_meta_2020-07-17_21:15/tensorboard does not exist, creating it instead!


In [6]:
df_train_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsTrain, header=0, sep='\t')
df_test_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsTest, header=0, sep='\t')
df_val_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsVal, header=0, sep='\t')

df_train_['2_way_label'] = df_train_['2_way_label'].apply(lambda x: np.array(x))
df_test_['2_way_label'] = df_test_['2_way_label'].apply(lambda x: np.array(x))
df_val_['2_way_label'] = df_val_['2_way_label'].apply(lambda x: np.array(x))

df_train = df_train_
df_test = df_test_
df_val = df_val_

# df_train = df_train[:15000]
# df_test = df_test[:15000]
# df_val = df_val[:15000]

df_train = checkDataframe(df_train, GLOBAL_BATCH_SIZE)
df_val = checkDataframe(df_val, GLOBAL_BATCH_SIZE)
df_test = checkDataframe(df_test, GLOBAL_BATCH_SIZE)


In [7]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_model_dir, "vocab.txt"))

all_title_data = FakeDetectionDataTrainVal(df_train, df_val, tokenizer, [0,1], MAX_SEQUENCE_LENGTH)

all_title_test = FakeDetectionDataTest(df_test, tokenizer, [0,1], MAX_SEQUENCE_LENGTH)


train_title_x = all_title_data.train_x
train_title_val_x = all_title_data.val_x
test_title_x = all_title_test.test_x


557056it [02:03, 4507.04it/s]
57344it [00:12, 4519.16it/s]


max seq_len 687


57344it [00:12, 4502.60it/s]


max seq_len 128


In [8]:
train_seq = DataSequenceTitleMeta(df_train, train_title_x, GLOBAL_BATCH_SIZE)

test_seq = DataSequenceTitleMeta(df_test, test_title_x, GLOBAL_BATCH_SIZE)

val_seq = DataSequenceTitleMeta(df_val, train_title_val_x, GLOBAL_BATCH_SIZE)

STEP_SIZE_TRAIN = len(df_train) // GLOBAL_BATCH_SIZE
STEP_SIZE_TEST = len(df_test) // GLOBAL_BATCH_SIZE
STEP_SIZE_VAL = len(df_val) // GLOBAL_BATCH_SIZE

In [9]:
# from tensorflow.keras.layers import Concatenate, Dense, Dropout, GlobalAveragePooling2D, Input, Lambda
# def buildConcatModelTitleMeta(model_bert_title, model_meta, NUM_CLASS):
#     model_bert_title_output = model_bert_title.get_layer('bert_output_layer_768').output
#     model_meta_output = model_meta.get_layer('final_output').output
    
#     # Build new models, without softmax
#     model_bert_title = Model(model_bert_title.inputs, model_bert_title_output)    
#     model_meta = Model(model_meta.inputs, model_meta_output)
    
#     x = Dense(128, activation = 'relu')(model_bert_title.output)
#     x = Dense (32, activation = 'relu')(x)
#     x = Dense (6, activation = 'relu')(x)
#     concatenate = Concatenate()([model_bert_title.output, x]) # Fusion
#     x = Dense(12, activation = 'relu')(concatenate)
#     x = Dropout(0.4)(x)
#     output = Dense(NUM_CLASS, activation='softmax', name='final_multimodal_output')(x)
#     model_concat = Model([model_bert_title.input, model_meta.input], output)
#     return model_concat

In [10]:
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    # Preparing:
    model_bert_title, model_bert_title_custom_layer = create_text_model(max_seq_len=MAX_SEQUENCE_LENGTH, bert_ckpt_file=bert_ckpt_file, bert_config_file= bert_config_file,NUM_CLASS=NUM_CLASS, overwriteLayerAndEmbeddingSize=False, isPreTrained=True,  pathToBertModelWeights=pathToBertModelTitle, isTrainable=RETRAIN_WHOLE_MODEL)
    model_meta, model_meta_custom_layer = create_model_meta(NUM_CLASS, 2, True, pathToMetaModel, isTrainable=RETRAIN_WHOLE_MODEL)
    
    model_concat = buildConcatModelTitleMeta(model_bert_title, model_meta, 2)
    
    if loadPretrained:
        model_concat.load_weights(ModelTitleMetaWeightsPath)

    if verbose:
        print('--------------Text Title MODEL --------------')
        model_bert_title.summary()
        print('-------------- Meta MODEL --------------')
        model_meta.summary()
        print('--------------CONCAT MODEL --------------')
        model_concat.summary()
        for l in model_concat.layers:
            print(l.name, l.trainable)
    
    
    model_concat.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
          optimizer=optimizer,
          metrics=['accuracy'])
    
    history = model_concat.fit(train_seq,
                    steps_per_epoch=STEP_SIZE_TRAIN,epochs=EPOCHS, 
                    validation_data=val_seq,
                    validation_steps=STEP_SIZE_VAL, 
                    callbacks=callbacks_list,
                    use_multiprocessing=False,
                    verbose=TF_VERBOSE,
                    workers=12
                   )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
bert shape (None, 128, 768)
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, 

In [11]:
%%capture

plot_model(model_concat, to_file=os.path.join(checkpointDir,'multiple_inputs_image_text_meta_pretrained.png'), show_shapes=True, dpi=600, expand_nested=False)
plot_model(model_concat, to_file=os.path.join(checkpointDir,'multiple_inputs_image_text_meta_pretrained_nested.png'), show_shapes=True, dpi=600, expand_nested=True)

In [12]:
# # For later model loading
# modelFile = '/home/armin/repos/FKD-Dataset/011_checkpoints/model_concat_title_image_2020-07-15_13:16/weights-improvement-02-0.91.hdf5'

# model_concat.load_weights(modelFile)

In [13]:
plotTimesPerEpoch(callbacks_list)

It took so long to train on one epoch: [20, 20, 20, 20, 20, 20, 20, 20, 20, 20] minutes


In [14]:
calcAccConcatModel(model_concat, val_seq, GLOBAL_BATCH_SIZE, 'Val')
calcAccConcatModel(model_concat, test_seq, GLOBAL_BATCH_SIZE, 'Test')

14/14 [==============================] - 92s 7s/step
Val Accuracy is 0.8813825334821429
14/14 [==============================] - 92s 7s/step
Test Accuracy is 0.881591796875


In [15]:
telegram_send_message(f'-----------------DONE-----------------')
print('done')

done
